# Max cut problem

Given a graph $G=(V, E)$, which has $|V|=n$ vertices and $|E|=m$ edges, introduce variable $y_{i}$ for every vertex $i \in[n]$. Say, $y_{i}=1$ if the vertex is assigned to $S_{1}$ and $y_{i}=-1$ if it is assigned to $S_{2}$. Then the task is to maximize the number of edges between $S_{1}$ and $S_{2}$.

If the edge $i, j$ is part of the cut (between $S_{1}$ and $\left.S_{2}\right), y_{i} y_{j}=-1$ otherwise it is $+1$. Then $\sum_{(i, j) \in E} \dfrac{1-y_{i} y_{j}}{2}$ counts the number of edges in the cut. Hence, the following integer program gives us the maximum cut.
$$
\begin{aligned}
& \max \sum_{(i, j) \in E} \dfrac{1-y_{i} y_{j}}{2} \\
\text { s.t. } \quad & y_{i} \in\{-1,1\} \quad \forall i
\end{aligned}
$$

The integer programming formulation was taken from [Lecture 10: Approximation Algorithm for Max Cut](https://www.cse.iitk.ac.in/users/rmittal/prev_course/s18/reports/10gw.pdf), section 1.1

Graph samples G1-G5 with edge weights equal to 1 were taken from http://web.stanford.edu/~yyye/yyye/Gset/

Reference result values were taken from the article "Breakout Local Search for the Max-Cut problem", table 2 

In [17]:
from gurobipy import Model, GRB, quicksum
import gurobipy as gp
import pandas as pd
import urllib
import os

In [18]:
# change directory to working directory
os.chdir("/Users/bacla/gurobi max_cut/")

# download graph samples
url = 'http://web.stanford.edu/~yyye/yyye/Gset/'
for i in range(1,6):
    url1 = url+'G'+f'{i}'
    urllib.request.urlretrieve(url1,f'G{i}.txt')

In [19]:
results = {}
results['sample'] = []
results['objval'] = []
models = {}

for i in range(1,6):
    sample = f'G{i}.txt'
    #retrieve number of nodes n and edges m
    with open(sample) as file:
        n, m = file.readline().split()
        n, m = int(n), int(m)

    df = pd.read_csv(sample, sep=' ', header=None, 
                     names=['1st vertice', '2nd vertice', 'weight'], skiprows=1)

    V = [i for i in range(1, n+1)]  # vertices
    E = [(i, j) for i, j in zip(df['1st vertice'], df['2nd vertice'])]  # edges

    model = Model('maxcut')

    y = model.addVars(V,
                      lb=-1, ub=1,
                      vtype=GRB.INTEGER,
                      name='y')

    model.addConstrs(y[i]**2 == 1 for i in range(1, 801))  # y = -1 or 1
    model.update()

    model.modelSense = GRB.MAXIMIZE
    model.setObjective(quicksum(1-y[i]*y[j] for i, j in E)/2)

    model.setParam('OutputFlag', False)

    model.Params.MIPGap = 0.1
    model.Params.TimeLimit = 60

    model.optimize()

    results['sample'].append(f'G{i}')
    results['objval'].append(model.objVal)

    models[f'G{i}'] = model

The results of the optimization are close or equal to best-known results reported in the literature and the most contribution in optimization was made by **gurobi presolve**

In [20]:
results_df = pd.DataFrame.from_dict(results)

results_df['results from article'] = [11624,11620,11622,11646,11631]
results_df[r'$\delta$'] = results_df['results from article']-results_df['objval']
results_df

,sample,objval,results from article,$\delta$
0,G1,11624.0,11624,0.0
1,G2,11620.0,11620,0.0
2,G3,11612.0,11622,10.0
3,G4,11641.0,11646,5.0
4,G5,11631.0,11631,0.0


Next, I tried to implement SDP relaxation and rounding from section 2 of [Lecture 10: Approximation Algorithm for Max Cut](https://www.cse.iitk.ac.in/users/rmittal/prev_course/s18/reports/10gw.pdf) and failed due to lack of memory

In [13]:
sample = 'G1.txt'
with open(sample) as file:
    n, m = file.readline().split()
    n, m = int(n), int(m)

df = pd.read_csv(sample, sep=' ', header=None, names=['1st vertice', '2nd vertice', 'weight'], skiprows=1)

V = [i for i in range(1, n+1)]  # vertices
E = [(i, j) for i, j in zip(df['1st vertice'], df['2nd vertice'])] # edges

model = Model('maxcut')
y = model.addMVar(shape=(n, n),
                  lb=-1, ub=1,
                  vtype=GRB.CONTINUOUS)
norm = model.addVar(lb=1,
                    ub=1,
                    vtype=GRB.INTEGER)
model.addConstrs((norm == gp.norm(y[i], 2)) for i in range(n))  # ||y_i|| = 1
model.update()

model.modelSense = GRB.MAXIMIZE
model.setObjective(quicksum(
    1-quicksum(y[i-1, k]*y[j-1, k] for k in range(n))
    for i, j in E)/2)

In [15]:
model.setParam('OutputFlag', True)
model.Params.NonConvex = 2
model.Params.MIPGap = 0.05
model.Params.TimeLimit = 600

model.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 640001 columns and 0 nonzeros
Model fingerprint: 0x8ad6337e
Model has 15340800 quadratic objective terms
Model has 800 general constraints
Variable types: 640000 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns (presolve time = 73s) ...
Presolve removed 0 rows and 1 columns
Presolve time: 72.83s
Presolved: 32602401 rows, 16620801 columns, 111225601 nonzeros
Presolved model has 15980800 bilinear constraint(s)
Variable types: 16620801 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 199.24 seconds (17.28 work units)
Thread count was 1 (of 8 available proce

GurobiError: Out of memory

In [40]:
#remove graph samples from local machine
print('Do you want to remove sample files?')
print('yes/[no]')
do_remove = input()
if do_remove == 'yes':
    for i in range(1,6):
        os.remove(f'G{i}.txt')
        print(f'G{i}.txt removed')
    print('All samples removed')

Do you want to remove sample files?
yes/[no]
